In [ ]:
import requests
import json
from requests_oauthlib import OAuth1
from util import create_oauth_token

# Ayudantía 12

Para realizar esta actividad deben tener una cuenta en trello. 

## Autentificaciones
Vamos a utilizar un servicio que requiere autentificarse. 
### Para obtener sus keys de aplicación entren al sitio https://trello.com/app-key
Les darán un api key y una api secret. Pongan eso en las variables de abajo

In [ ]:
TRELLO_API_KEY = 
TRELLO_API_SECRET = 

### Para obtener los permisos del usuario ejecuten esta línea

In [ ]:
oauth_token = create_oauth_token(key = TRELLO_API_KEY, secret = TRELLO_API_SECRET)

In [ ]:
TRELLO_OAUTH_TOKEN = oauth_token['oauth_token']
TRELLO_OAUTH_TOKEN_SECRET = oauth_token['oauth_token_secret']

### Esta variable debeb utilizarla para autentificarse y hacer modificaciones en sus tableros

In [ ]:
auth = OAuth1(client_key=TRELLO_API_KEY, client_secret=TRELLO_API_SECRET,
     resource_owner_key=TRELLO_OAUTH_TOKEN, resource_owner_secret=TRELLO_OAUTH_TOKEN_SECRET)

## Hagamos requests :)

In [ ]:
url = 'https://api.trello.com/1' # URL del servicio

###Links útiles
#### API de TRELLO
https://developers.trello.com/advanced-reference

##### Librería Requests
http://docs.python-requests.org/en/latest/
### Obteniendo información de la web
####Queremos obtener:
- Las listas del tablero
- Los labels del tablero
- Las tarjetas del tablero

### Opción 1

In [ ]:
url_lists = url + '/boards'+ '/nTepCisO' + '/lists'
response = requests.get(url_lists, auth=auth)
lists = response.json()

In [ ]:
url_labels = url + '/boards' + '/nTepCisO' + '/labels'
response = requests.get(url_labels, auth=auth)
labels = response.json()

In [ ]:
url_cards = url + '/boards' + '/nTepCisO' + '/cards'
response = requests.get(url_cards, auth=auth)
cards = response.json()

### Opción 2

In [ ]:
response = requests.request('GET', url_board + '/lists',auth=auth)
lists = response.json()

In [ ]:
response = requests.request('GET', url_board + '/labels',auth=auth)
labels = response.json()

In [ ]:
response = requests.request('GET', url_board + '/cards',auth=auth)
cards = response.json()

#### Con esa información queremos saber 

1 . El nombre de la lista a la que pertenece cada una de las tarjetas 

In [ ]:
def find_list(lists, id_):
    for l in lists:
        if l['id'] == id_:
            return l['name']
        
for c in cards:
    list_name = find_list(lists, c['idList'])
    print(c['name'] + ' está en la lista ' + list_name)

2 . Los nombres de los labels utilizados en cada lista

In [ ]:
def find_label(labels, id_):
    for l in labels:
        if l['id'] == id_:
            return l['name']

labels_in_lists = {}

for l in lists:
    cards_in_list = list(filter(lambda x: x['idList']==l['id'], cards))
    labels_in_lists.update({l['id']: []})
    for c in cards_in_list:
        labels_in_lists[l['id']].extend(c['idLabels'])

for id_, l in labels_in_lists.items():
    list_name = find_list(lists, id_)
    print('En lista ' + list_name + ' están los siguientes labels: ')
    for label_id in l:
        label_name = find_label(labels, label_id)
        print(' ' + label_name)
    

### Enviando información a la web
Qué pasa si queremos agregar tarjetas a la web?

In [ ]:
def add_card(name, list_id, labels_id ,desc=None, due=None):
    post_args={'name': name, 'idList': list_id, 'desc': desc, 'idLabels': labels_id, 'due': due}
    data = json.dumps(post_args) # El contenido del post 
    headers = {}
    headers['Content-Type'] = 'application/json; charset=utf-8' # El header de la request
    response = requests.request('POST', url + '/cards',data=data, headers=headers, auth=auth)
    # response = requests.post(url + '/cards, headers=header, data=data, auth=auth)
    return response.status_code

In [ ]:
add_card('Hola', '5648d215c2f82b0b11ede108', ['5648d1e019ad3a5dc2274ce4'])